# Note, this *only* works for observing things in the future. Dealing with the pointing corrections in the analysis is not supported here.





# First thing to do, make a copy of this notebook under File->Make a Copy
---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
tstart = '2017-07-17T00:00:00'
tend = '2017-07-19T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

---
# Step 1, get the nominal RA/Dec Position for Jupiter at the start.

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('jup310.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
jupiter, earth = planets['jupiter'], planets['earth']
astrometric = earth.at(t).observe(jupiter)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 jupiter_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/jupiter_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



../orbit_engine/jupiter_198to200.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/jupiter_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2017-07-17 00:23:30,2017-07-17 01:19:29
1,2017-07-17 02:00:10,2017-07-17 02:56:09
2,2017-07-17 03:36:50,2017-07-17 04:32:49
3,2017-07-17 05:13:30,2017-07-17 06:09:28
4,2017-07-17 06:50:09,2017-07-17 07:46:08


# Use SkyField to get the location of Jupiter for each orbit:

This puts the output into the provided text file. This shows the *aim* time that was used to determine the pointing. You should slew while the source is occulted.

In [4]:
from nustar_planning import jupiter_planning
from imp import reload
outfile = 'jupiter_{0}_{1}_pointing.txt'.format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )
jupiter_planning.position(orbits, load_path ='../../data',
                 parallax_correction=True,
                 outfile=outfile)
print("Output is stored in: {}".format(outfile))

Output is stored in: jupiter_198_200_pointing.txt
